# Robot Main

In [1]:
%register 'serial:///dev/serial0'
%connect robot-stm

Connected to robot-stm @ serial:///dev/serial0


In [18]:
%register 'serial:///dev/serial0'
%connect robot-stm
%rsync
%softreset

%%host

import nest_asyncio
nest_asyncio.apply()

from iot_device.pydevice import Pydevice
from serial import Serial
from array import array
from struct import unpack
import asyncio, time

def repl_no_follow(cmd):
    with Serial('/dev/serial0', 115200, timeout=0.5, write_timeout=2, exclusive= True) as serial:
        pyb = Pydevice(serial)
        pyb.enter_raw_repl()
        pyb.exec_raw_no_follow(cmd)
        

        
class Robot:
    
    def __init__(self):
        self.baudrate = 1000000
        
    async def wait_for_msg(self):
        print("wait_for_msg")
        serial = self.serial
        while True:
            if serial.in_waiting:
                t = serial.read(1)
                # print("type", t)
                if t == b's':
                    print("state", array('i', serial.read(8*4)))
                elif t == b'm':
                    size = unpack('H', serial.read(2))[0]
                    print(f"> {serial.read(size).decode()}")
                else:
                    print("*** unkown type", t)
            else:
                await asyncio.sleep(0.1)
                
    async def main(self):
        serial = self.serial
        msg_task = asyncio.create_task(self.wait_for_msg())
        serial.write(b'v')
        serial.write(array('b', [100, 0]))
        print("enable motors")
        serial.write(b'e')
        await asyncio.sleep(10)
        serial.write(b'v')
        serial.write(array('b', [30, 0]))
        await asyncio.sleep(3)
        print("disable motors")
        serial.write(b'd')
        await asyncio.sleep(1)
        msg_task.cancel()
        print("main exits")
    
    def __enter__(self):
        # start program on MCU
        with Serial('/dev/serial0', 115200, timeout=0.5, write_timeout=2, exclusive= True) as serial:
            pyb = Pydevice(serial)
            pyb.enter_raw_repl()
            pyb.exec_raw_no_follow(
                f"import main_\n"
                f"main_.start({self.baudrate})\n"
            )
        time.sleep(1)
        self.serial = Serial('/dev/serial0', self.baudrate, timeout=0.5, write_timeout=2, exclusive= True)
        # start tasks
        asyncio.run(self.main())
        print("enter returns")
        return self
    
    def __exit__(self, *args):
        self.serial.close()
        self.reset_mcu()
        
    def reset_mcu(self):
        print("RESET_MCU")
        from gpiozero import LED
        from time import sleep
        with LED(21) as nrst:
            nrst.off()
            sleep(0.1)
            nrst.on()
            sleep(0.1)

with Robot() as robot:
    print("robot", robot)
    
print("DONE")

Connected to robot-stm @ serial:///dev/serial0
Directories match

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!   softreset ...     !!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

enable motors
wait_for_msg
disable motors
main exits
enter returns
robot <__main__.Robot object at 0x71ecab70>
RESET_MCU
DONE


In [17]:
import pyb

print("A")
uart = pyb.UART(3, 1000000, timeout=500)
a = 5
uart.init(115200)
# pyb.repl_uart(uart)

print("reset", a)

A
reset 5


In [17]:
import uasyncio as asyncio
import pyb

async def blink():
    led = pyb.LED(1)
    for i in range(3):
        print("t", i)
        led.toggle()
        await asyncio.sleep(0.2)

async def forever():
    while True:
        print("f", )
        await asyncio.sleep(0.15)

async def main():
    print("A")
    f = asyncio.create_task(forever()) 
    await asyncio.gather(blink())
    print("B")

asyncio.run(main())

print("DONE")

f
A
f
t 0
f
f
t 1
f
f
t 2
f
f
f
f
B
DONE


In [17]:
print(5)

5
